In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import time

In [2]:
base_url = catalog.load('params:oai_fetch_options.base_url')
context = catalog.load('params:oai_fetch_options.context')
env = 'dev'


                    INFO     Loading data from params:oai_fetch_options.base_url                ]8;id=573879;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=150091;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_fetch_options.context                 ]8;id=626451;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=272439;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [3]:
base_url

'https://ri.conicet.gov.ar/oai'

La función **get_oai_records()** también se usa para recuperar items en otro nodo

In [4]:
def get_oai_records(base_url):
    start_time = time.time()

    response = requests.get(base_url)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Sleeping for {elapsed_time:.2f} seconds")
    time.sleep(elapsed_time)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        return None


In [5]:
def fetch_oai_sets(base_url, context, env):

    resumption_token = 0
    all_sets = []

    iteration_limit = 2
    iteration_count = 0

    while True:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        params = f'/{context}?verb=ListSets&resumptionToken=////{resumption_token}'
        url = base_url + params

        print(f"Consultando: {url}")

        response = get_oai_records(url)
        if not response:
            break

        xml_content = response.text
        root = ET.fromstring(xml_content)
        ns = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

        sets_data = []
        for set_elem in root.findall('.//oai:set', ns):
            set_spec = set_elem.find('oai:setSpec', ns).text if set_elem.find('oai:setSpec', ns) is not None else None
            set_name = set_elem.find('oai:setName', ns).text if set_elem.find('oai:setName', ns) is not None else None
            sets_data.append({'setSpec': set_spec, 'setName': set_name})

        if not sets_data:
            print("No se encontraron más sets.")
            break

        all_sets.extend(sets_data)
        resumption_token += 100  # avanzar manualmente
        iteration_count += 1

    df_sets = pd.DataFrame(all_sets)
    return df_sets


In [6]:
df = fetch_oai_sets(base_url, context, env)


Consultando: https://ri.conicet.gov.ar/oai/request?verb=ListSets&resumptionToken=////0
Sleeping for 0.35 seconds
Consultando: https://ri.conicet.gov.ar/oai/request?verb=ListSets&resumptionToken=////100
Sleeping for 0.36 seconds


In [7]:
df

,setSpec,setName
0,snrd,Sistema Nacional de Repositorios Digitales
1,com_11336_73,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
2,com_11336_116,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
3,com_11336_169,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
4,com_11336_184,AREA DE INFLUENCIA CENTRO CIENTÍFICO TECNOLÓGI...
...,...,...
195,com_11336_174,IIB - INSTITUTO DE INVESTIGACIONES BIOLÓGICAS
196,com_11336_167,IIB-INTECH - INSTITUTO DE INVESTIGACIONES BIOT...
197,com_11336_108933,IIBIO - INSTITUTO DE INVESTIGACIONES BIOTECNOL...
198,com_11336_104,IIBYT - INSTITUTO DE INVESTIGACIONES BIOLÓGICA...
